<a href="https://colab.research.google.com/github/HananeMaghlazi/P2_Education/blob/main/Lemonde_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import des librairies
import re,os,glob,time
from pathlib import Path
import pandas as pd 
import numpy as np
import math
from numpy import arange
import seaborn as sns
import matplotlib.pyplot as plt
# Pour les warnings
import warnings
warnings.filterwarnings('ignore')
# Pour le traitement de texte
import nltk
from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import (
    sent_tokenize,
    word_tokenize,
    wordpunct_tokenize,
)
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import cluster, metrics
from sklearn import manifold, decomposition
import pyarrow.parquet as pq

In [2]:
#Modification des paramètres d'affichages des colonnes, lignes et largeurs des colonnes pour obtenir le 
#plus d'informations
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', None)
pd.set_option('float_format',lambda x: "%.4f" % x)

Installation nltk

In [3]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
# Monter le drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Lecture des fichiers parquets

In [16]:
df = pd.read_parquet('/content/drive/MyDrive/climat')
df.head()

,url,news,news_publication,publication_name,publication_language,news_publication_date,news_title,news_keywords,image,image_loc,image_caption,sitemap,etag,sitemap_last_modified,sitemap_size_mb,download_date,media,section,changefreq,news_access,image_title,lastmod,news_genres,priority,download_date_last,media_type
64318,https://www.lemonde.fr/disparitions/article/2022/12/31/barbara-walters-legende-de-la-television-americaine-est-morte-a-93-ans_6156166_3382.html,None,None,Le Monde,fr,2022-12-31 10:00:26,"Barbara Walters, légende de la télévision américaine, est morte à 93 ans",None,None,https://img.lemde.fr/2022/12/31/307/0/2852/1426/1440/720/60/0/2aefa65_6ecc96ea1de147f3a8667f420f8ca74a-0-4957a456686b4458a6dda828748cb38c.jpg,"Barbara Walters, le 7 octobre 2014, à l’université de Harvard.",https://www.lemonde.fr/sitemap_news.xml,"""597c9bf8fc520caad7935c42820685ff""",2023-01-01 01:47:06+00:00,0.1021,2023-01-01 01:48:03,lemonde,"[disparitions, article]",None,None,None,2022-12-31 09:00:26+00:00,None,NaN,2023-01-02 01:28:02,webpress
64319,https://www.lemonde.fr/international/article/2022/12/31/entre-la-syrie-et-la-turquie-un-premier-pas-officiel-vers-un-rapprochement_6156164_3210.html,None,None,Le Monde,fr,2022-12-31 14:43:55,"Entre la Syrie et la Turquie, un premier pas officiel vers un rapprochement",None,None,https://img.lemde.fr/2022/12/30/273/0/3277/1638/1440/720/60/0/eab1351_1672390913048-ap22203813165385.jpg,"Le ministre russe de la défense, Sergueï Choïgou, à gauche, et son homologue turc, Hulusi Akar, se serrent la main lors d’une réunion à Istanbul, en Turquie, le vendredi 22 juillet 2022.",https://www.lemonde.fr/sitemap_news.xml,"""597c9bf8fc520caad7935c42820685ff""",2023-01-01 01:47:06+00:00,0.1021,2023-01-01 01:48:03,lemonde,"[international, article]",None,None,None,2022-12-31 13:43:55+00:00,None,NaN,2023-01-02 01:28:02,webpress
64320,https://www.lemonde.fr/argent/article/2022/12/31/passoires-thermiques-timbres-smic-gaz-carburant-ce-qui-change-le-1er-janvier-pour-le-budget-des-menages_6156161_1657007.html,None,None,Le Monde,fr,2022-12-31 06:01:39,"Passoires thermiques, timbres, smic, gaz, carburant : ce qui change le 1er janvier pour le budget des ménages",None,None,https://img.lemde.fr/2022/12/29/0/0/12000/6000/1440/720/60/0/c2dc5d0_1672331682749-080-hl-hpassarello-1045946.jpg,"Devant une borne de rechargement du passe Navigo, dans le métro parisien, en mars 2020.",https://www.lemonde.fr/sitemap_news.xml,"""597c9bf8fc520caad7935c42820685ff""",2023-01-01 01:47:06+00:00,0.1021,2023-01-01 01:48:03,lemonde,"[argent, article]",None,None,None,2022-12-31 05:01:39+00:00,None,NaN,2023-01-02 01:28:02,webpress
64321,https://www.lemonde.fr/idees/article/2022/12/31/en-guyane-la-transition-energetique-ne-se-fera-pas-en-niant-les-droits-des-peuples-autochtones-et-en-saccageant-la-foret-amazonienne_6156169_3232.html,None,None,Le Monde,fr,2022-12-31 08:00:20,"En Guyane, « la transition énergétique ne se fera pas en niant les droits des peuples autochtones et en saccageant la forêt amazonienne »",None,None,https://img.lemde.fr/2021/05/24/118/0/1600/800/1440/720/60/0/bc641a8_712450648-09-tgy7363-tanguystoeckle-preview.jpg,"Dans la réserve naturelle nationale des Nouragues, en Guyane.",https://www.lemonde.fr/sitemap_news.xml,"""597c9bf8fc520caad7935c42820685ff""",2023-01-01 01:47:06+00:00,0.1021,2023-01-01 01:48:03,lemonde,"[idees, article]",None,None,None,2022-12-31 07:00:20+00:00,None,NaN,2023-01-02 01:28:02,webpress
64322,https://www.lemonde.fr/idees/article/2022/12/31/mort-de-mahsa-amini-la-seule-maniere-de-mettre-fin-a-la-crise-en-iran-est-d-en-finir-avec-l-intervention-des-religieux-dans-la-politique_6156168_3232.html,None,None,Le Monde,fr,2022-12-31 08:00:10,Mort de Mahsa Amini : « La seule manière de mettre fin à la crise en Iran est d’en finir avec l’intervention des religieux dans la politique »,None,None,https://img.lemde.fr/2022/10/08/291/0/3500/1750/1440/720/60/0/73acbf5_fw1-iran-women-0918-11.jpg,None,https://www.lemonde.fr

In [4]:
df.shape

(2912, 26)

In [17]:
#Suppresssion des colonnes nulls et sans utilité
print("colonnes avant supression",df.shape[1])
col_to_drop=["priority","changefreq","image","image_title","news","news_access","news_genres","news_keywords",
             "news_publication","publication_language","image_loc","sitemap","media_type","media_type",
             "sitemap_size_mb","sitemap","media","publication_name"]
df.drop(col_to_drop,axis=1,inplace=True)
print("colonnes après supression",df.shape[1])

colonnes avant supression 26
colonnes après supression 10


In [23]:
df.head()

,url,news_publication_date,news_title,image_caption,etag,sitemap_last_modified,download_date,section,lastmod,download_date_last
64318,https://www.lemonde.fr/disparitions/article/20...,2022-12-31 10:00:26,"Barbara Walters, légende de la télévision amér...","Barbara Walters, le 7 octobre 2014, à l’univer...","""597c9bf8fc520caad7935c42820685ff""",2023-01-01 01:47:06+00:00,2023-01-01 01:48:03,"[disparitions, article]",2022-12-31 09:00:26+00:00,2023-01-02 01:28:02
64319,https://www.lemonde.fr/international/article/2...,2022-12-31 14:43:55,"Entre la Syrie et la Turquie, un premier pas o...","Le ministre russe de la défense, Sergueï Choïg...","""597c9bf8fc520caad7935c42820685ff""",2023-01-01 01:47:06+00:00,2023-01-01 01:48:03,"[international, article]",2022-12-31 13:43:55+00:00,2023-01-02 01:28:02
64320,https://www.lemonde.fr/argent/article/2022/12/...,2022-12-31 06:01:39,"Passoires thermiques, timbres, smic, gaz, carb...",Devant une borne de rechargement du passe Navi...,"""597c9bf8fc520caad7935c42820685ff""",2023-01-01 01:47:06+00:00,2023-01-01 01:48:03,"[argent, article]",2022-12-31 05:01:39+00:00,2023-01-02 01:28:02
64321,https://www.lemonde.fr/idees/article/2022/12/3...,2022-12-31 08:00:20,"En Guyane, « la transition énergétique ne se f...",Dans la réserve naturelle nationale des Nourag...,"""597c9bf8fc520caad7935c42820685ff""",2023-01-01 01:47:06+00:00,2023-01-01 01:48:03,"[idees, article]",2022-12-31 07:00:20+00:00,2023-01-02 01:28:02
64322,https://www.lemonde.fr/idees/article/2022/12/3...,2022-12-31 08:00:10,Mort de Mahsa Amini : « La seule manière de me...,None,"""597c9bf8fc520caad7935c42820685ff""",2023-01-01 01:47:06+00:00,2023-01-01 01:48:03,"[idees, article]",2022-12-31 07:00:10+00:00,2023-01-02 01:28:02


In [18]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [19]:
# Stop words
stop_w = list(set(stopwords.words("french"))) + [
    "[",
    "]",
    ",",
    ".",
    ":",
    "?",
    "(",
    ")",
]

In [20]:
df.news_title.iloc[0]

'Barbara Walters, légende de la télévision américaine, est morte à 93\xa0ans'

Fonctions du prétraitement

In [21]:
# Tokenizer
def tokenizer_fct(sentence):
    # print(sentence)
    sentence_clean = (
        sentence.replace("-", " ").replace("+", " ").replace("/", " ").replace("#", " ")
    )
    word_tokens = word_tokenize(sentence_clean)
    return word_tokens



def stop_word_filter_fct(list_words):
    filtered_w = [w for w in list_words if not w in stop_w]
    filtered_w2 = [w for w in filtered_w if len(w) > 2]
    #filtered_w3 = [w for w in filtered_w2 if not w in list_unique_words]
    return filtered_w2


# lower case et alpha
def lower_start_fct(list_words):
    lw = [
        w.lower().strip()
        for w in list_words
        if (not w.startswith("@"))
        and (not w.startswith("#"))
        and (not w.startswith("http"))
    ]
    return lw


# Lemmatizer (base d'un mot)


def lemma_fct(list_words):
    lemmatizer = WordNetLemmatizer()
    lem_w = [lemmatizer.lemmatize(w) for w in list_words]
    return lem_w

In [22]:
# Fonction de préparation du texte pour le bag of words (Countvectorizer et Tf_idf)
def transform_bow_fct(desc_text):
    word_tokens = tokenizer_fct(desc_text)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    transf_desc_text = " ".join(lw)
    return transf_desc_text


# Fonction de préparation du texte pour le bag of words avec lemmatization
def transform_bow_lem_fct(desc_text):
    word_tokens = tokenizer_fct(desc_text)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    lem_w = lemma_fct(lw)
    transf_desc_text = " ".join(lem_w)
    return transf_desc_text

Application sur la colonne new-title

In [23]:
#df["news_title"] = df["news_title"].astype(str)
df["sentence_bow"] = df["news_title"].apply(lambda x: transform_bow_fct(x))
df["sentence_bow_lem"] = df["news_title"].apply(lambda x: transform_bow_lem_fct(x))

Extraire la section

In [24]:
df["section"]=df["section"].str[0]
l_sect = list(set(df["section"]))
print("Sections : ", l_sect)
y_sect_num = [(1 - l_sect.index(df.iloc[i]["section"])) for i in range(len(df))]

Sections :  ['mondephilatelique', 'series', 'm-styles', 'le-monde-evenements', 'sports-de-glisse', 'sante', 'podcasts', 'energies', 'police-justice', 'critique-litteraire', 'videos', 'afrique', 'les-envies-du-monde', 'campus', 'education', 'argent', 'sciences', 'football', 'disparitions', 'cinema', 'tennis', 'pixels', 'les-recettes-du-monde', 'le-monde-passe-a-table', 'economie', 'm-perso', 'sport', 'emploi', 'televisions-radio', 'le-monde-des-religions', 'musiques', 'blog-mediateur', 'culture', 'politique', 'societe', 'guides-d-achat', 'actualite-medias', 'smart-cities', 'un-si-proche-orient', 'planete', 'm-le-mag', 'rugby', 'livres', 'climat', 'idees', 'le-jazz-et-la-salsa', 'international', 'les-decodeurs']


In [39]:
len(l_sect)

48

In [25]:
df["length_bow"] = df["sentence_bow"].apply(lambda x: len(word_tokenize(x)))
print("max length bow : ", df["length_bow"].max())

df["length_bow_lem"] = df["sentence_bow_lem"].apply(lambda x: len(word_tokenize(x)))
print("max length bow lem : ", df["length_bow_lem"].max())


max length bow :  20
max length bow lem :  20


In [35]:
df.head()

,url,news_publication_date,news_title,image_caption,etag,sitemap_last_modified,download_date,section,lastmod,download_date_last,sentence_bow,sentence_bow_lem,length_bow,length_bow_lem
64318,https://www.lemonde.fr/disparitions/article/2022/12/31/barbara-walters-legende-de-la-television-americaine-est-morte-a-93-ans_6156166_3382.html,2022-12-31 10:00:26,"Barbara Walters, légende de la télévision américaine, est morte à 93 ans","Barbara Walters, le 7 octobre 2014, à l’université de Harvard.","""597c9bf8fc520caad7935c42820685ff""",2023-01-01 01:47:06+00:00,2023-01-01 01:48:03,"[disparitions, article]",2022-12-31 09:00:26+00:00,2023-01-02 01:28:02,barbara walters légende télévision américaine morte ans,barbara walter légende télévision américaine morte an,7,7
64319,https://www.lemonde.fr/international/article/2022/12/31/entre-la-syrie-et-la-turquie-un-premier-pas-officiel-vers-un-rapprochement_6156164_3210.html,2022-12-31 14:43:55,"Entre la Syrie et la Turquie, un premier pas officiel vers un rapprochement","Le ministre russe de la défense, Sergueï Choïgou, à gauche, et son homologue turc, Hulusi Akar, se serrent la main lors d’une réunion à Istanbul, en Turquie, le vendredi 22 juillet 2022.","""597c9bf8fc520caad7935c42820685ff""",2023-01-01 01:47:06+00:00,2023-01-01 01:48:03,"[international, article]",2022-12-31 13:43:55+00:00,2023-01-02 01:28:02,entre syrie turquie premier officiel vers rapprochement,entre syrie turquie premier officiel vers rapprochement,7,7
64320,https://www.lemonde.fr/argent/article/2022/12/31/passoires-thermiques-timbres-smic-gaz-carburant-ce-qui-change-le-1er-janvier-pour-le-budget-des-menages_6156161_1657007.html,2022-12-31 06:01:39,"Passoires thermiques, timbres, smic, gaz, carburant : ce qui change le 1er janvier pour le budget des ménages","Devant une borne de rechargement du passe Navigo, dans le métro parisien, en mars 2020.","""597c9bf8fc520caad7935c42820685ff""",2023-01-01 01:47:06+00:00,2023-01-01 01:48:03,"[argent, article]",2022-12-31 05:01:39+00:00,2023-01-02 01:28:02,passoires thermiques timbres smic gaz carburant change 1er janvier budget ménages,passoires thermiques timbre smic gaz carburant change 1er janvier budget ménages,11,11
64321,https://www.lemonde.fr/idees/article/2022/12/31/en-guyane-la-transition-energetique-ne-se-fera-pas-en-niant-les-droits-des-peuples-autochtones-et-en-saccageant-la-foret-amazonienne_6156169_3232.html,2022-12-31 08:00:20,"En Guyane, « la transition énergétique ne se fera pas en niant les droits des peuples autochtones et en saccageant la forêt amazonienne »","Dans la réserve naturelle nationale des Nouragues, en Guyane.","""597c9bf8fc520caad7935c42820685ff""",2023-01-01 01:47:06+00:00,2023-01-01 01:48:03,"[idees, article]",2022-12-31 07:00:20+00:00,2023-01-02 01:28:02,guyane transition énergétique fera niant droits peuples autochtones saccageant forêt amazonienne,guyane transition énergétique fera niant droits peuples autochtones saccageant forêt amazonienne,11,11
64322,https://www.lemonde.fr/idees/article/2022/12/31/mort-de-mahsa-amini-la-seule-maniere-de-mettre-fin-a-la-crise-en-iran-est-d-en-finir-avec-l-intervention-des-religieux-dans-la-politique_6156168_3232.html,2022-12-31 08:00:10,Mort de Mahsa Amini : « La seule manière de mettre fin à la crise en Iran est d’en finir avec l’intervention des religieux dans la politique »,None,"""597c9bf8fc520caad7935c42820685ff""",2023-01-01 01:47:06+00:00,2023-01-01 01:48:03,"[idees, article]",2022-12-31 07:00:10+00:00,2023-01-02 01:28:02,mort mahsa amini seule manière mettre fin crise iran finir intervention religieux politique,mort mahsa amini seule manière mettre fin crise iran finir intervention religieux politique,13,13


In [50]:
# Calcul Tsne, détermination des clusters et calcul ARI entre vrais sections et n° de clusters
def ARI_fct(features):
    time1 = time.time()
    num_labels = len(l_sect) # nombre de sections
    tsne = manifold.TSNE( # paramètres tsne
        n_components=2,
        perplexity=30,
        #n_iter=2000,
        init="random",
        learning_rate=200,
        random_state=42,
    )
    X_tsne = tsne.fit_transform(features)

    # Détermination des clusters à partir des données après Tsne
    cls = cluster.KMeans(n_clusters=num_labels, random_state=42)# initialisation kmeans
    cls.fit(X_tsne) # fiter le kmeans sur données réduites tsne
    ARI = np.round(metrics.adjusted_rand_score(y_sect_num, cls.labels_), 4)# calcul ari entre vrai sections et resultats clustering kmeans
    time2 = np.round(time.time() - time1, 0)
    print("ARI : ", ARI, "time : ", time2)

    return ARI, X_tsne, cls.labels_


In [26]:
# création du bag of words (CountVectorizer et Tf-idf)

cvect = CountVectorizer(stop_words=stop_w, max_df=0.95, min_df=1)
ctf = TfidfVectorizer(stop_words=stop_w, max_df=0.95, min_df=1)

feat = "sentence_bow_lem"
cv_fit = cvect.fit(df[feat])
ctf_fit = ctf.fit(df[feat])

cv_transform = cvect.transform(df[feat])
ctf_transform = ctf.transform(df[feat])

In [32]:
cvect

CountVectorizer(max_df=0.95,
                stop_words=['dans', 'auras', 'au', 'ayant', 'avec', 'fussent',
                            'me', 'j', 'ta', 'aurai', 'le', 'moi', 'étantes',
                            'vous', 'il', 'c', 'un', 'ce', 'qu', 'toi', 'étais',
                            'auront', 'ayante', 'soyez', 'as', 'eut', 'que',
                            'on', 'qui', 'sommes', ...])

In [36]:
num_labels = len(l_sect)
num_labels

48

In [33]:
# Calcul Tsne, détermination des clusters et calcul ARI entre vrais sections et n° de clusters
def ARI_fct(features):
    time1 = time.time()
    num_labels = len(l_sect) # nombre de sections
  
    # Détermination des clusters à partir des données 
    cls = cluster.KMeans(n_clusters=num_labels, random_state=42)# initialisation kmeans
    cls.fit(features) # fiter le kmeans 
    ARI = np.round(metrics.adjusted_rand_score(y_sect_num, cls.labels_), 4)# calcul ari entre vrai sections et resultats clustering kmeans
    time2 = np.round(time.time() - time1, 0)
    print("ARI : ", ARI, "time : ", time2)

    return ARI, cls.labels_


In [34]:
# Résultats sans réduction
print("CountVectorizer : ")
print("-----------------")
ARI,  labels = ARI_fct(cv_transform)
print()
print("Tf-idf : ")
print("--------")
ARI, labels = ARI_fct(ctf_transform)

CountVectorizer : 
-----------------
ARI :  -0.003 time :  3.0

Tf-idf : 
--------
ARI :  0.0044 time :  4.0


In [37]:
# Résultats avec réduction
print("CountVectorizer : ")
print("-----------------")
ARI, X_tsne_cv, labels = ARI_fct(cv_transform)
print()
print("Tf-idf : ")
print("--------")
ARI, X_tsne_tf, labels = ARI_fct(ctf_transform)

CountVectorizer : 
-----------------
ARI :  0.0309 time :  31.0

Tf-idf : 
--------
ARI :  0.052 time :  40.0
